<a href="https://colab.research.google.com/github/prateekmanral011/hackathon_spg_slb/blob/main/nearby_faults_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab / Jupyter notebook: Structural features per well-depth sample
# Install required packages (run once in Colab)
!pip install -q pandas numpy shapely scipy geopandas

# ---- Imports ----
import os, glob, re
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point, MultiLineString
from shapely.ops import linemerge
from scipy.spatial import cKDTree
from collections import defaultdict
from math import sqrt

# ---- USER CONFIG (update these paths if needed) ----
# If files are in Google Drive, mount drive and set paths under /content/drive/MyDrive/...
WELL_FILE = "/content/All_wells_dat.csv"   # path to your uploaded well trajectory CSV
STICK_FOLDER = "/content/sticks/"                      # folder where you upload all stick CSVs
OUTPUT_FILE = "/content/well_structural_features_near_faults.csv"

# If WELL_FILE doesn't contain depth-sampled rows, set SAMPLE_FLAG_USE_LOGS=False and provide Top/Bottom TVD
SAMPLE_FLAG_USE_LOGS = True
SAMPLE_STEP = 1.0   # step (m) for sampling if not logs

# KDTree/search params
KD_NEAREST_K = 8

# Radii (meters) for density/length stats
R_list = [350]

# -----------------------
# Helper functions
# -----------------------
def safe_numeric(s):
    return pd.to_numeric(s, errors='coerce')

def canonical_fault_id_from_filename(fname):
    base = os.path.splitext(os.path.basename(fname))[0]
    base = re.sub(r'(_converted|_conv)$', '', base, flags=re.I)
    base = re.sub(r'(_|-)\d+[a-zA-Z]?$','', base)
    base = re.sub(r'(_|-)[a-zA-Z]$','', base)
    return base

def infer_fault_type_from_filename(fname):
    n = fname.lower()
    if 'main' in n: return 'main'
    if 'branch' in n or 'branched' in n: return 'branch'
    if 'closing' in n: return 'closing'
    if 'truncat' in n or 'truncating' in n: return 'truncating'
    if 'boundary' in n or 'bound' in n: return 'boundary'
    return 'other'

def load_stick_csv(fp):
    """
    Load stick CSV and return dict {name,fault_id,fault_type,xy,z,linestring}
    NOTE: flips Z sign to positive downward (because many stick files have negative Z).
    """
    df = pd.read_csv(fp, header=0)
    cols = list(df.columns)
    # find first three numeric columns
    numeric_cols = []
    for c in cols:
        if safe_numeric(df[c]).notna().sum() > 1:
            numeric_cols.append(c)
        if len(numeric_cols) >= 3:
            break
    if len(numeric_cols) < 3:
        # fallback: try to coerce first three columns
        raise ValueError(f"Could not infer XYZ numeric columns in {fp}. Columns: {cols}")
    xcol, ycol, zcol = numeric_cols[:3]
    x = safe_numeric(df[xcol]).values
    y = safe_numeric(df[ycol]).values
    z = safe_numeric(df[zcol]).values
    mask = (~np.isnan(x)) & (~np.isnan(y)) & (~np.isnan(z))
    x = x[mask]; y = y[mask]; z = z[mask]
    if len(x) < 2:
        return None
    # Flip Z sign to match well TVD positive-down if sticks have negative Z
    z = -z
    xy = np.vstack([x, y]).T
    ls = LineString(xy.tolist())
    fname = os.path.basename(fp)
    fid = canonical_fault_id_from_filename(fname)
    ftype = infer_fault_type_from_filename(fname)
    return {'name': fname, 'fault_id': fid, 'fault_type': ftype, 'xy': xy, 'z': z, 'linestring': ls}

def nearest_point_on_segment(px, py, x0,y0,x1,y1, z0, z1):
    vx = x1 - x0; vy = y1 - y0
    wx = px - x0; wy = py - y0
    seg_len2 = vx*vx + vy*vy
    if seg_len2 == 0:
        t = 0.0
    else:
        t = (wx*vx + wy*vy) / seg_len2
        if t < 0.0: t = 0.0
        elif t > 1.0: t = 1.0
    nx = x0 + t*vx; ny = y0 + t*vy; nz = z0 + t*(z1 - z0)
    return nx, ny, nz, t

# -----------------------
# 1) Load/prepare wells_samples
# -----------------------
print("Loading wells from:", WELL_FILE)
wdf = pd.read_csv(WELL_FILE)
print("Columns in well file:", list(wdf.columns))
print("Sample rows:")
display(wdf.head(6))

if SAMPLE_FLAG_USE_LOGS:
    if not {'WellID','X','Y','TVD'}.issubset(set(wdf.columns)):
        # If your column names differ (case, or 'md' present), try to map common names automatically:
        rename_map = {}
        cols_l = [c.lower() for c in wdf.columns]
        if 'wellid' in cols_l:
            rename_map[wdf.columns[cols_l.index('wellid')]] = 'WellID'
        elif 'well' in cols_l:
            rename_map[wdf.columns[cols_l.index('well')]] = 'WellID'
        if 'x' in cols_l:
            rename_map[wdf.columns[cols_l.index('x')]] = 'X'
        elif 'utmx' in cols_l:
            rename_map[wdf.columns[cols_l.index('utmx')]] = 'X'
        if 'y' in cols_l:
            rename_map[wdf.columns[cols_l.index('y')]] = 'Y'
        elif 'utmy' in cols_l:
            rename_map[wdf.columns[cols_l.index('utmy')]] = 'Y'
        if 'tvd' in cols_l:
            rename_map[wdf.columns[cols_l.index('tvd')]] = 'TVD'
        elif 'tvdss' in cols_l:
            rename_map[wdf.columns[cols_l.index('tvdss')]] = 'TVD'
        if rename_map:
            print("Auto-renaming columns:", rename_map)
            wdf = wdf.rename(columns=rename_map)
    # After potential renaming, validate
    if not {'WellID','X','Y','TVD'}.issubset(set(wdf.columns)):
        raise ValueError("WELL_FILE must contain WellID,X,Y,TVD columns when SAMPLE_FLAG_USE_LOGS=True")
    wells_samples = wdf[['WellID','X','Y','TVD','md']].copy()
else:
    # Use Top_TVD and Base_TVD to sample depths
    if not {'WellID','X','Y','Top_TVD','Base_TVD'}.issubset(set(wdf.columns)):
        raise ValueError("WELL_FILE must contain WellID,X,Y,Top_TVD,Base_TVD when SAMPLE_FLAG_USE_LOGS=False")
    rows = []
    for _, r in wdf.iterrows():
        top = float(r['Top_TVD']); bot = float(r['Base_TVD'])
        if bot < top:
            bot, top = top, bot
        depths = np.arange(top, bot + 0.0001, SAMPLE_STEP)
        for d in depths:
            rows.append({'WellID': r['WellID'], 'X': r['X'], 'Y': r['Y'], 'TVD': d})
    wells_samples = pd.DataFrame(rows)

print("Number of well samples:", len(wells_samples))

# -----------------------
# 2) Load stick files
# -----------------------
print("Loading stick CSVs from:", STICK_FOLDER)
stick_files = glob.glob(os.path.join(STICK_FOLDER, "*.csv"))
if len(stick_files) == 0:
    raise ValueError(f"No stick CSV files found in {STICK_FOLDER}. Upload them to that folder.")
print("Found", len(stick_files), "CSV files (sticks). Loading...")

sticks = []
for fp in stick_files:
    try:
        s = load_stick_csv(fp)
        if s:
            sticks.append(s)
    except Exception as e:
        print("Warning: could not load", fp, ":", e)

if len(sticks) == 0:
    raise ValueError("No valid stick files loaded.")

print("Loaded", len(sticks), "stick objects (may correspond to fewer unique faults).")

# Build mapping fault_id -> stick indices
fault_to_stick_idxs = defaultdict(list)
for si, s in enumerate(sticks):
    fault_to_stick_idxs[s['fault_id']].append(si)

# fault type map
fault_type_map = {fid: sticks[idxs[0]]['fault_type'] for fid, idxs in fault_to_stick_idxs.items()}

# Merge per-fault geometry (combine sticks of same fault)
fault_geoms = {}
for fid, sidxs in fault_to_stick_idxs.items():
    lines = [sticks[i]['linestring'] for i in sidxs]
    try:
        merged = linemerge(lines)
    except Exception:
        merged = MultiLineString(lines)
    fault_geoms[fid] = merged

# Build flattened vertex array and index_map (vertex -> (stick_idx, vertex_idx))
all_vertices = []
index_map = []
for si, s in enumerate(sticks):
    for vi, (xx,yy) in enumerate(s['xy']):
        all_vertices.append((xx,yy))
        index_map.append((si,vi))
all_vertices = np.array(all_vertices)
kdtree = cKDTree(all_vertices)
print("KDTree built on", len(all_vertices), "vertices.")

# -----------------------
# 3) Compute features per well sample
# -----------------------
results = []
n_samples = len(wells_samples)
print("Computing structural features for", n_samples, "samples...")

for idx, r in wells_samples.iterrows():
    px = float(r['X']); py = float(r['Y']); ptvd = float(r['TVD'])
    # nearest 3D: query K nearest vertices
    K = min(KD_NEAREST_K, len(all_vertices))
    dists, vidx = kdtree.query([px,py], k=K)
    if np.isscalar(vidx):
        vidx = [vidx]
    best = {'dist3d': 1e12, 'stick_idx': None, 'nearest_z': None, 'nx': None, 'ny': None, 'fault_type': None, 'fault_id': None}
    for v in vidx:
        si, vi = index_map[int(v)]
        s = sticks[si]
        nverts = s['xy'].shape[0]
        segs = []
        if vi-1 >= 0: segs.append((vi-1, vi))
        if vi+1 < nverts: segs.append((vi, vi+1))
        for a,b in segs:
            x0,y0 = s['xy'][a]; x1,y1 = s['xy'][b]
            z0 = s['z'][a]; z1 = s['z'][b]
            nx, ny, nz, t = nearest_point_on_segment(px,py, x0,y0, x1,y1, z0, z1)
            ddx = px - nx; ddy = py - ny; ddz = ptvd - nz
            dist3d = sqrt(ddx*ddx + ddy*ddy + ddz*ddz)
            if dist3d < best['dist3d']:
                best.update({'dist3d': dist3d, 'stick_idx': si, 'nearest_z': nz, 'nx': nx, 'ny': ny,
                             'fault_type': s['fault_type'], 'fault_id': s['fault_id'], 'stick_name': s['name']})
    # fallback brute force if none found
    if best['stick_idx'] is None:
        min_d = 1e12
        for si, s in enumerate(sticks):
            for vi, (xx,yy) in enumerate(s['xy']):
                dz = ptvd - s['z'][vi]
                dd = sqrt((px-xx)**2 + (py-yy)**2 + dz*dz)
                if dd < min_d:
                    min_d = dd
                    best.update({'dist3d': dd, 'stick_idx': si, 'nearest_z': s['z'][vi], 'nx': xx, 'ny': yy,
                                 'fault_type': s['fault_type'], 'fault_id': s['fault_id'], 'stick_name': s['name']})

    # Compute counts and lengths by fault (unique fault ids) within each radius
    row_out = {'WellID': r['WellID'], 'X': px, 'Y': py, 'TVD': ptvd,'MD': r['md'],

               'dist_3D_to_nearest_fault': best['dist3d'],
               'nearest_fault_type': best['fault_type'],
               'nearest_fault_id': best['fault_id'],
               'nearest_stick_name': best.get('stick_name', None),
               'nearest_fault_z': best.get('nearest_z', None),
               'vertical_offset_to_fault': (ptvd - best.get('nearest_z')) if best.get('nearest_z') is not None else None}

      # Ensure we have the complete list of fault types
    fault_types = sorted(set(fault_type_map.values()))

    for R in R_list:
        # find vertices inside radius -> candidate stick indices -> candidate fault ids
        vids = kdtree.query_ball_point([px,py], r=R)
        candidate_stick_idxs = set(index_map[ii][0] for ii in vids) if len(vids)>0 else set()
        candidate_fault_ids = set(sticks[sid]['fault_id'] for sid in candidate_stick_idxs) if len(candidate_stick_idxs)>0 else set()

        # now compute length inside buffer & per-type stats
        buf = Point(px,py).buffer(R)
        total_len = 0.0
        per_type_count = {tt: 0 for tt in fault_types}
        per_type_length = {tt: 0.0 for tt in fault_types}

        for fid in candidate_fault_ids:
            geom = fault_geoms[fid]
            inter = geom.intersection(buf)
            if inter.is_empty:
                ln = 0.0
            else:
                if hasattr(inter, 'geoms'):
                    ln = sum(g.length for g in inter.geoms)
                else:
                    ln = inter.length
            if ln > 0:
                total_len += ln
                ftype = fault_type_map.get(fid, 'other')
                per_type_count[ftype] += 1
                per_type_length[ftype] += ln

        # write standardized outputs (zeros when none)
        row_out[f'fault_count_within_{R}m'] = len(candidate_fault_ids)
        row_out[f'fault_length_within_{R}m'] = total_len

        for ttype in fault_types:
            row_out[f'fault_count_{ttype}_within_{R}m'] = per_type_count.get(ttype, 0)



    results.append(row_out)

# -----------------------
# Save results
# -----------------------
outdf = pd.DataFrame(results)
outdf.to_csv(OUTPUT_FILE, index=False)
print("Saved structural features to:", OUTPUT_FILE)
display(outdf.head())


Loading wells from: /content/All_wells_dat.csv
Columns in well file: ['WellID', 'X', 'Y', 'TVD', 'md']
Sample rows:


,WellID,X,Y,TVD,md
0,A10,456979.0637,6782712.412,1499.878992,1499.878992
1,A10,456979.1646,6782712.394,1499.991677,1500.031292
2,A10,456979.2657,6782712.377,1500.104433,1500.183692
3,A10,456979.3666,6782712.359,1500.217110,1500.335992
4,A10,456979.4676,6782712.342,1500.329784,1500.488292
5,A10,456979.5687,6782712.324,1500.442675,1500.640892


Number of well samples: 50651
Loading stick CSVs from: /content/sticks/
Found 22 CSV files (sticks). Loading...
Loaded 22 stick objects (may correspond to fewer unique faults).
KDTree built on 704 vertices.
Computing structural features for 50651 samples...
Saved structural features to: /content/well_structural_features_near_faults.csv


,WellID,X,Y,TVD,MD,dist_3D_to_nearest_fault,nearest_fault_type,nearest_fault_id,nearest_stick_name,nearest_fault_z,vertical_offset_to_fault,fault_count_within_350m,fault_length_within_350m,fault_count_boundary_within_350m,fault_count_branch_within_350m,fault_count_closing_within_350m,fault_count_main_within_350m,fault_count_other_within_350m,fault_count_truncating_within_350m
0,A10,456979.0637,6782712.412,1499.878992,1499.878992,311.355352,closing,Closing_Fault_South_2 (1),Closing_Fault_South_2 (1)_converted.csv,1758.093509,-258.214517,2,3763.396120,0,0,1,1,0,0
1,A10,456979.1646,6782712.394,1499.991677,1500.031292,311.245691,closing,Closing_Fault_South_2 (1),Closing_Fault_South_2 (1)_converted.csv,1758.093509,-258.101832,2,3763.306592,0,0,1,1,0,0
2,A10,456979.2657,6782712.377,1500.104433,1500.183692,311.136550,closing,Closing_Fault_South_2 (1),Closing_Fault_South_2 (1)_converted.csv,1758.093509,-257.989076,2,3763.210571,0,0,1,1,0,0
3,A10,456979.3666,6782712.359,1500.217110,1500.335992,311.026968,closing,Closing_Fault_South_2 (1),Closing_Fault_South_2 (1)_converted.csv,1758.093509,-257.876399,2,3763.121043,0,0,1,1,0,0
4,A10,456979.4676,6782712.342,1500.329784,1500.488292,310.917974,closing,Closing_Fault_South_2 (1),Closing_Fault_South_2 (1)_converted.csv,1758.093509,-257.763725,2,3763.025219,0,0,1,1,0,0


**Seperating all 15 wells for merging it to porosity , permeability , Fluvial fecies and gamma ray values i.e with las file data**

In [3]:
import pandas as pd
import os

# Path to your combined CSV
big_csv = "/content/well_structural_features_near_faults.csv"

# Output folder
out_folder = "/content/wells_split/"
os.makedirs(out_folder, exist_ok=True)

# Load data
df = pd.read_csv(big_csv)

# Check what well IDs exist
print("Unique WellIDs in file:", df['WellID'].unique())

# List of wells you want separate files for
target_wells = ["A10","A15","A16","B1","B2","B4","B8","B9",
                "C1","C2","C3","C4","C5","C6","C7"]

# Loop and write each subset
for well in target_wells:
    sub = df[df['WellID'] == well]
    if sub.empty:
        print(f"⚠️ No rows found for {well}, skipping...")
        continue
    out_path = os.path.join(out_folder, f"{well}_features.csv")
    sub.to_csv(out_path, index=False)
    print(f"✅ Wrote {len(sub)} rows for {well} -> {out_path}")

print("\nAll done. Separate CSVs are in:", out_folder)


Unique WellIDs in file: ['A10' 'A15' 'A16' 'B1' 'B2' 'B4' 'B8' 'B9' 'C1' 'C2' 'C3' 'C4' 'C5' 'C6'
 'C7']
✅ Wrote 6011 rows for A10 -> /content/wells_split/A10_features.csv
✅ Wrote 1805 rows for A15 -> /content/wells_split/A15_features.csv
✅ Wrote 2297 rows for A16 -> /content/wells_split/A16_features.csv
✅ Wrote 8268 rows for B1 -> /content/wells_split/B1_features.csv
✅ Wrote 7612 rows for B2 -> /content/wells_split/B2_features.csv
✅ Wrote 8268 rows for B4 -> /content/wells_split/B4_features.csv
✅ Wrote 3839 rows for B8 -> /content/wells_split/B8_features.csv
✅ Wrote 8268 rows for B9 -> /content/wells_split/B9_features.csv
✅ Wrote 466 rows for C1 -> /content/wells_split/C1_features.csv
✅ Wrote 546 rows for C2 -> /content/wells_split/C2_features.csv
✅ Wrote 582 rows for C3 -> /content/wells_split/C3_features.csv
✅ Wrote 756 rows for C4 -> /content/wells_split/C4_features.csv
✅ Wrote 534 rows for C5 -> /content/wells_split/C5_features.csv
✅ Wrote 597 rows for C6 -> /content/wells_split/C